In [16]:
import ee
import pandas as pd
from pathlib import Path
from datetime import datetime, timezone
import os
import re

from google.auth.transport.requests import AuthorizedSession

ee.Authenticate()  #  or !earthengine authenticate --auth_mode=gcloud
session = AuthorizedSession(ee.data.get_persistent_credentials())

## Create empty image collections as target
Create empty image collection with earthengine CLI
Purge contents if delete or clean repopulation needed
List COGs in GCS folder

```
cd /data/gis/raster_base/Alaska/AKVegMap/disturbance

conda activate ee

earthengine create collection projects/akveg-map/assets/disturbance/nbac_annual_v20240530
earthengine create collection projects/akveg-map/assets/disturbance/nbac_annual_v20240530_lf_3338
earthengine create collection projects/akveg-map/assets/disturbance/nbac_annual_v20240530_102001

for i in `earthengine ls projects/akveg-map/assets/disturbance/nbac_annual_v20240530`; do earthengine rm $i; done

gsutil ls gs://akveg-data/disturbance/nbac/v20240530/*.tif > nbac_annual_v20240530_on_gcs.txt
```

In [17]:
import json
# from urllib.parse import urlparse
import os
from pprint import pprint


In [18]:
# TODO Fix date extraction from filename to work for different filename formats
# Parts are zero-based
def load_gcs_cogs_to_collection(cogs, project_folder, collection, year_part, version_part):
    # Request body as a dictionary.
    for cog in cogs['tif']:
      fileOnly = os.path.split(cog)[1]
      # print(fileOnly)

      cogName = fileOnly[:-4]
      print(cogName)
      
      # parts = fileOnly.split('_')
      # Split on both underscore and period
      parts = re.split(r'[_\.]', fileOnly)
      print(parts)
      
      year = f"{parts[year_part]}"
      year = int(year)
      print(year)

      version = f"{parts[version_part]}"
      print(version)

      start_dt = datetime(year, 1, 1)
      start_dt = f"{start_dt:%Y-%m-%dT%H:%M:%SZ}"  # Format string for ISO 8601 + Z
      end_dt = datetime(year + 1, 1, 1)
      end_dt = f"{end_dt:%Y-%m-%dT%H:%M:%SZ}"  # Format string for ISO 8601 + Z
      print(start_dt, end_dt)
        
      request = {
        'type': 'IMAGE',
        'gcs_location': {
          'uris': cog
        },
        'properties': {
            'version': version,
            #   'source': 'https://code.earthengine.google.com/067b10ee56537817756a3177a9138aee',
            # 'yyyymmdd_hhmmss': yyyymmdd_hhmmss
        },
        'startTime': start_dt,#'2023-01-01T00:00:00.000000000Z',
        'endTime': end_dt,#'2024-01-01T00:00:00.000000000Z',
      }

      # pprint(json.dumps(request))

      # A folder (or ImageCollection) name and the new asset name.
      asset_id = collection+'/'+cogName
      # print(project_folder)
      # print(asset_id)
        
      url = 'https://earthengine.googleapis.com/v1alpha/projects/{}/assets?assetId={}'
      print(url)

      response = session.post(
        url = url.format(project_folder, asset_id),
        data = json.dumps(request)
      )

      pprint(json.loads(response.content))
    print('done')


## Run it

In [14]:
# earthengine create collection projects/akveg-map/assets/disturbance/nbac_annual_v20240530_102001
# gsutil ls gs://akveg-data/disturbance/nbac/v20240530/cog_102001/*.tif > nbac_annual_102001_v20240530_on_gcs.txt
# for i in `earthengine ls projects/akveg-map/assets/disturbance/nbac_annual_v20240530_102001`; do earthengine rm $i; done
# earthengine ls projects/akveg-map/assets/disturbance/nbac_annual_v20240530_102001 | xargs -P 20 -n 1 earthengine rm

project_folder = 'akveg-map'
collection = 'disturbance/nbac_annual_v20240530_102001'
cogs = pd.read_csv('/data/gis/raster_base/Alaska/AKVegMap/disturbance/nbac_annual_102001_v20240530_on_gcs.txt', header=None,names=['tif'])

load_gcs_cogs_to_collection(cogs, project_folder, collection, 4, 3)


nbac_30m_102001_v20240530_1972
['nbac', '30m', '102001', 'v20240530', '1972', 'tif']
1972
v20240530
1972-01-01T00:00:00Z 1973-01-01T00:00:00Z
https://earthengine.googleapis.com/v1alpha/projects/{}/assets?assetId={}
{'bands': [{'dataType': {'precision': 'INT', 'range': {'max': 65535}},
            'grid': {'affineTransform': {'scaleX': 30,
                                         'scaleY': -30,
                                         'translateX': -2282865,
                                         'translateY': 3779295},
                     'crsWkt': 'PROJCS["Canada_Albers_Equal_Area_Conic", \n'
                               '  GEOGCS["NAD83", \n'
                               '    DATUM["North American Datum 1983", \n'
                               '      SPHEROID["GRS 1980", 6378137.0, '
                               '298.257222101, AUTHORITY["EPSG","7019"]], \n'
                               '      TOWGS84[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, '
                               '0.0], \

In [20]:
# cd /data/gis/raster_base/Alaska/AKVegMap/disturbance
# earthengine create collection projects/akveg-map/assets/disturbance/nbac_annual_v20240530_lf_3338
# gsutil ls gs://akveg-data/disturbance/nbac/v20240530/cog_lf_3338/*.tif > nbac_annual_lf_3338_v20240530_on_gcs.txt
# for i in `earthengine ls projects/akveg-map/assets/disturbance/nbac_annual_v20240530_lf_3338`; do earthengine rm $i; done
# earthengine ls projects/akveg-map/assets/disturbance/nbac_annual_v20240530_lf_3338 | xargs -P 20 -n 1 earthengine rm

project_folder = 'akveg-map'
collection = 'disturbance/nbac_annual_v20240530_lf_3338'
cogs = pd.read_csv('/data/gis/raster_base/Alaska/AKVegMap/disturbance/nbac_annual_lf_3338_v20240530_on_gcs.txt', header=None,names=['tif'])

load_gcs_cogs_to_collection(cogs, project_folder, collection, 4, 3)


nbac_30m_3338_v20240530_1972
['nbac', '30m', '3338', 'v20240530', '1972', 'tif']
1972
v20240530
1972-01-01T00:00:00Z 1973-01-01T00:00:00Z
https://earthengine.googleapis.com/v1alpha/projects/{}/assets?assetId={}
{'bands': [{'dataType': {'precision': 'INT', 'range': {'max': 65535}},
            'grid': {'affineTransform': {'scaleX': 30,
                                         'scaleY': -30,
                                         'translateX': -2175045,
                                         'translateY': 2383725},
                     'crsCode': 'EPSG:3338',
                     'dimensions': {'height': 65974, 'width': 125270}},
            'id': 'B0',
            'missingData': {'values': [0]},
            'pyramidingPolicy': 'MEAN'}],
 'cloudStorageLocation': {'uris': ['gs://akveg-data/disturbance/nbac/v20240530/cog_lf_3338/nbac_30m_3338_v20240530_1972.tif']},
 'endTime': '1973-01-01T00:00:00Z',
 'geometry': {'coordinates': [[172.2480478944905, 53.02311792076253],
                